In [1]:
import os
import sys
from pathlib import Path
ROOT= Path.cwd()
sys.path.append(str(ROOT/"src"))


In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
%pip install --upgrade pip
%pip install -r requirements.txt
import box, yaml, sys
print ("box",box.__version__, "->", sys.executable)

Note: you may need to restart the kernel to use updated packages.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Note: you may need to restart the kernel to use updated packages.
box 6.0.2 -> /Users/fatemehshahlaei/Documents/mlproject /venv/bin/python


In [4]:
from pathlib import Path
import sys, importlib.util, os

# 1) Find project root (where requirements.txt is)
ROOT = Path.cwd()
while ROOT != ROOT.parent and not (ROOT/"requirements.txt").exists():
    ROOT = ROOT.parent

print("PROJECT_ROOT =", ROOT)

# 2) Make sure src/ is on sys.path
SRC = ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

# 3) Quick diagnostics
print("src on path? ->", str(SRC) in sys.path)
print("mlProject exists? ->", (SRC/"mlProject").is_dir())
print("spec ->", importlib.util.find_spec("mlProject"))

PROJECT_ROOT = /Users/fatemehshahlaei/Documents/end to end ml project with mlops
src on path? -> True
mlProject exists? -> True
spec -> ModuleSpec(name='mlProject', loader=<_frozen_importlib_external.SourceFileLoader object at 0x1105def10>, origin='/Users/fatemehshahlaei/Documents/end to end ml project with mlops/src/mlProject/__init__.py', submodule_search_locations=['/Users/fatemehshahlaei/Documents/end to end ml project with mlops/src/mlProject'])


In [5]:
%pip install -r "{ROOT / 'requirements.txt'}"

Obtaining file:///Users/fatemehshahlaei/Documents/end%20to%20end%20ml%20project%20with%20mlops/research (from -r /Users/fatemehshahlaei/Documents/end to end ml project with mlops/requirements.txt (line 15))
ERROR: file:///Users/fatemehshahlaei/Documents/end%20to%20end%20ml%20project%20with%20mlops/research (from -r /Users/fatemehshahlaei/Documents/end to end ml project with mlops/requirements.txt (line 15)) does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
Note: you may need to restart the kernel to use updated packages.


In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

from mlProject.logger import logger

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [8]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

In [ ]:
from mlProject.utils.common import read_yaml
from mlProject.constants import CONFIG_FILE_PATH

config = read_yaml(CONFIG_FILE_PATH)
print(config)